In [1]:
from pyMez import *

Importing pyMez, this should take roughly 30 seconds
Importing Code.DataHandlers.GeneralModels
It took 2.401 s to import Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.HTMLModels
It took 0.355 s to import Code.DataHandlers.HTMLModels
Importing Code.DataHandlers.NISTModels
It took 8.632 s to import Code.DataHandlers.NISTModels
Importing Code.DataHandlers.TouchstoneModels
It took 0.018 s to import Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
It took 0.352 s to import Code.DataHandlers.XMLModels
Importing Code.DataHandlers.ZipModels
It took 0.04 s to import Code.DataHandlers.ZipModels
Importing Code.InstrumentControl.Experiments
It took 1.98 s to import Code.InstrumentControl.Experiments
Importing Code.InstrumentControl.Instruments
It took 0.03 s to import Code.InstrumentControl.Instruments
Importing Code.Utils.Names
It took 0.022 s to import Code.Utils.Names
It took 13.831 s to import all of the active modules


In [2]:
import visa

In [4]:
rm=visa.ResourceManager()

In [5]:
rm.list_resources()

(u'ASRL3::INSTR',
 u'ASRL10::INSTR',
 u'GPIB0::1::INSTR',
 u'GPIB0::7::INSTR',
 u'GPIB0::19::INSTR')